# Recursion and Memoization: Towards Dynamic Programming
We continue our discussion on memoization and illustrate how that can lead to a rewrite of the code so that it is no longer recursive.  The approach, called **Dynamic Programming**, essentially involves combining memoization and recursive problem solving process together to change the order of computation, so that results have already been memoized by the time they are needed. 

In [1]:
%config InteractiveShell.ast_node_interactivity="none"

In [2]:
!wget https://raw.githubusercontent.com/jamcoders/syllabus-resources-2023/main/week3/lecs/boaz_utils.ipynb
%run "boaz_utils.ipynb"

# Fibonacci Revisited
Let us look back at how we defined our memoized recursive `fib` function.

In [26]:
fib_memory = {}   # using "global variables" like this is not a great programming practice

def fib_memo(n):
    '''Compute fibonacci number if not already in memory'''
    if n in fib_memory:  
        return fib_memory[n]
    elif n < 2:
        ans = n
    else:
        ans = fib_memo(n - 1) + fib_memo(n - 2)
    fib_memory[n] = ans
    return ans

In [27]:
print(fib_memo(10))
print(fib_memory)

55
{1: 1, 0: 0, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13, 8: 21, 9: 34, 10: 55}


- When is the first time that a value gets stored in the dictionary? (Is it on the first call to `fib_memo`?)
- When is the first time that a value gets retrieved from the dictionary? (Assuming that the recursive calls are done in left to right order.)

**Idea:**
What if we were to change the order of computation so that we never tried to get the answer to a value that we didn't already have?

In [19]:
def dp_fib(n):
    fib_memory = {}
    # insert the base cases:
    fib_memory[0] = 0
    fib_memory[1] = 1
    # We have enough information to do 2, then 3, then 4, etc
    for i in range(2, n+1):  # we need nth value too
        fib_memory[i] = fib_memory[i-1] + fib_memory[i-2]
    return fib_memory[n]

In [20]:
print(dp_fib(10))

55


The above implementation is considered to be a **Dynamic Programming** implementation of the Fibonacci function.  When designing a DP approach, we need:

- The recursive rule
- The base cases
- A table initialised with the base cases
- A process to update the entries in the table in an iterative way (we will have to choose the order of inputs to update)

Instead of recursive calls, we will see references to the table.

# Counting Paths in a Grid
Suppose that we have a robot that starts in the top left corner (coordinate (0,0)) of a grid, and on each step, is allowed to move only a single square either to the right or downward.  We wish to count the number of paths that it may take to get to a given coordinate $(r, c)$ (row $r$, column $c$).

Let $p(r, c)$ denote the number of paths that end at cell $(r, c)$

Some values for $p(r, c)$ are shown in the grid below. (We can use these as test cases).

|p(r, c)| 0 | 1 | 2 | 3 |
|-------|---|---|---|---|
| **0**     | 1 | 1 | 1 | 1 |
| **1**     | 1 | 2 | 3 | 4 |
| **2**     | 1 | 3 | 6 | ? |

Let us develop a dynamic programming solution to computing $p(r, c)$.

## The Recursive Approach
- What is the recursive rule?
- What are the base cases?


### Recursive Rule:
**Observation:** To get to cell $(r, c)$, on the previous step, the robot had to be either at the cell above or the cell to the left.  So the total number of ways to get to $(r, c)$ must be the sum of the number of ways to get to either of those adjacent cells. 

$$p(r, c) = p(r-1, c) + p(r, c-1)$$

### Base Cases:
**Observations:**  
- The robot starts at $(0, 0)$ so there is exactly 1 way to get there.  ($p(0, 0) = 1$)
- Since the robot cannot go up, there is only 1 way to get to each cell in the topmost row. ($p(0, j) = 1$ for all $j \geq 0$.)
- Since the robot cannot go left, there is only 1 way to get to each cell in the leftmost column. ($p(i, 0) = 1$ for all $i \geq 0$)
- The robot cannot drive off the top edge, or the left edge, so there are 0 ways to get to a negative coordinate in either the row or the column. ($p(r, c) = 0$ if $r < 0$ or $c < 0$) 


In [28]:
def count_paths(r, c):
    if r == 0 or c == 0:
        return 1
    else:
        return count_paths(r-1, c) + count_paths(r, c-1)

In [33]:
print(count_paths(50, 50))

KeyboardInterrupt: 

In [14]:
def count_paths(r, c):
    if r < 0 or c < 0:
        return 0
    elif r == 0 or c == 0:
        return 1
    else:
        return count_paths(r-1, c) + count_paths(r, c-1)

In [15]:
print(count_paths(2, 2))

6


Now, we are ready to memoize, but first, we show how we can combine multiple inputs to make a single key for our dictionary: 

In [16]:
test_mem = {}
test_mem['1 2'] = 1
test_mem[(1,2)] = 3

print(test_mem['1 2'])
print(test_mem[(1, 2)])

1
3


In [34]:
def memo_count_paths(r, c):
    cp_memory = {}
    def helper(i, j):
        key = (i, j)
        if key in cp_memory:
            return cp_memory[key]
        else:
            if i == 0 or j == 0:
                ans = 1
            else:
                ans = helper(i-1, j) + helper(i, j-1)
            cp_memory[key] = ans
            return ans
    return helper(r, c)
            

In [37]:
print(memo_count_paths(50, 50))

100891344545564193334812497256


Now, let us try to rephrase this as a DP algorithm. We need to:
- Make a table of values of $p(r, c)$
- Create loops that will move in increasing order of $r$ and $c$
- Replace recursive calls above with table references

In [38]:
def dp_count_paths(row, col):
    cp_memory = {}
    cp_memory[(0, 0)] = 1
    # Initialise top row in memory
    for j in range(col+1):
        cp_memory[(0, j)] = 1
    # Initialise left column in memory
    for i in range(row + 1):
        cp_memory[(i, 0)] = 1
    # Now for the rest of the table (ref else of recursive impl)
    for r in range(1, row + 1):
        for c in range(1, col + 1):
            cp_memory[(r, c)] = cp_memory[(r-1, c)] + cp_memory[(r, c-1)]
    return cp_memory[(row, col)]

In [42]:
print(dp_count_paths(4, 4))

70


# Coin Change Variant
Let us revisit the coin change problem, but this time, we will ask for the **number of ways** to make change, rather than the smallest set of coins.

Let $c(n, S)$ represent the number of ways to make change for an amount $n$ using only the denominations available in $S$.

Example:  Say $S = \{5, 2, 1\}$ and $n=8$.  Then, we can make change in the following ways:
1.  5 + 1 + 1 + 1
1.  5 + 2 + 1
1.  2 + 2 + 2 + 2
1.  2 + 2 + 2 + 1 + 1
1.  2 + 2 + 1 + 1 + 1 + 1
1.  2 + 1 + 1 + 1 + 1 + 1 + 1
1.  1 + 1 + 1 + 1 + 1 + 1 + 1 + 1

So $c(8, \{5, 2, 1\}) = 7$.

- What is the recursive rule for $c(n, S)$? (Hint: think back to the discussion this morning)
- What are the right base cases?


## Recursive Rule(s)
**Observation:**
- Say $S = \{s_0, s_1, ...\}$
- The number of ways to make change for $n$ using $S$ is the sum of the number of ways to make change for $n$ using $s_0$ (the first element of $S$) plus the number of ways to make change without using it.
$$c(n, S) = c(n - s_0, S) + c(n, S - \{s_0\})$$

**Base Cases:**
- $c(0, S) = 1$ for each version of $S$ that arises
- $c(n, []) = 0$ for any $n > 0$

Can we go straight to the DP implementation?

Almost.  First we need to figure out how we will handle the keys in our memory that are based on function arguments. 

The second argument to our function is a list, which cannot be used as a key to a dictionary in Python. Let us encode list arguments as strings before including them in keys.  Since, we always remove only the first element when changing the list argument, the only possible list arguments that arise are slices that start at indexes 0, 1, ... and go all the way to the end of the list.

In [48]:
def mk_slice_strings(coins):
    result = []
    for i in range(len(coins) + 1):
        result.append(str(coins[i:]))
    return result

In [49]:
print(mk_slice_strings([20, 10, 5, 1]))

['[20, 10, 5, 1]', '[10, 5, 1]', '[5, 1]', '[1]', '[]']


In [55]:
def dp_make_change(n, coins):
    mc_memory = {}
    k = len(coins)
    # Stringify the list arguments that can arise
    lst_arg_strs = mk_slice_strings(coins)
    lst_arg_strs.reverse()  # Order we want for DP
    # A key will be a tuple (n, str(coins))
    
    # Fill in memory for n = 0
    for lst in lst_arg_strs:
        mc_memory[(0, lst)] = 1
    
    # Now fill in memory for S = []
    empty_list_str = lst_arg_strs[0]  # '[]'
    for i in range(1, n+1):
        mc_memory[(i, empty_list_str)] = 0
        
    # Complete rest of the memory
    for i in range(1, n+1):  # the current amount
        for j in range(k):   # the current start index for the slice of coins
            prev = lst_arg_strs[j]
            lst = lst_arg_strs[j+1]
            coin = coins[j]
            if i >= coin:
                mc_memory[(i, lst)] = mc_memory[(i - coin, lst)] + mc_memory[(i, prev)]
            else:
                mc_memory[(i, lst)] = mc_memory[(i, prev)]
                    
    return mc_memory[(n, lst_arg_strs[-1])]

In [54]:
print(dp_make_change(8, [5, 2, 1]))


7
